<a href="https://colab.research.google.com/github/jiimmyelly/Project-Capstone/blob/main/Interactive_Ploty_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash

In [ ]:
!pip install wget

In [ ]:
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv") 

'spacex_launch_dash.csv'

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

In [ ]:

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options='ALL', value='ALL', placeholder='Select a Launch Site here', searchable='True'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
             Input(component_id='site-dropdown',component_property='value'))

def pie(site):
    if site == "ALL":
        fig = px.pie(spacex_df, values= 'class', names='Launch Site', title='Success count for all launch site')
        return fig
    else:
        df_site_filtered = spacex_df[spacex_df['Launch Site']== site]
        df_site = df_site_filtered[df_site_filtered['Launch Site','class']]
        df_site_final = df_site.groupby(['Launch Site'], as_index=False)['class'].count()
        fig = px.pie(df_site_final, values='class', names= 'Launch Site', title='Success count for'+ site)
    return fig

In [ ]:

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def scatter(site, payload):
    low, high = (payload[0],payload[1])
    mask = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site == 'ALL':
        fig = px.scatter(mask, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Success count on Payload Mass for all sites')
        return fig
    else:
        mask_filtered = mask[mask['Launch Site'] == site]
        fig = px.scatter(mask_filtered, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Success count on Payload Mass for '+ site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Oct/2021 12:23:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:05] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1621065734.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:05] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1621065734.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:06] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1621065734.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:06] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_16_0m1621065734.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:06] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1621065734.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 12:24:06] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1621065734.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 1